In [ ]:
import requests
import time

locations = [
    "AXI", "CAM", "CON", "CRO", "DAR", "DER", "EST-L", "EST-S", "FUL", "GRA",
    "HER", "IRO", "JAC", "LAT", "MLT", "MOB", "THM", "URB", "WDC"
]

url = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3/query"

for loc in locations:
    params = {
        "where": f"report_location='{loc}'",
        "outFields": "report_location,report_tract_no,Tract_Name,tract_status_desc,Forester,tract_type_family,SaleType,latitude_dd,longitude_dd,Wthr_grd,PurchDate,harvest_status",
        "resultRecordCount": 2000,
        "f": "json",
        "returnGeometry": "false"
    }

    # Count request
    count_params = {
        "where": f"report_location='{loc}'",
        "returnCountOnly": "true",
        "f": "json"
    }

    count_resp = requests.get(url, params=count_params)
    count = count_resp.json().get("count", 0)

    # Timed request
    start = time.time()
    response = requests.get(url, params=params)
    elapsed = round((time.time() - start) * 1000)  # in milliseconds

    print(f'{{ "report_location": "{loc}", "count": {count}, "time": "{elapsed:,} ms" }}')


In [4]:
import requests
import time
import json

def run_query(include_harvest_status=True):
    locations = [
        "AXI", "CAM", "CON", "CRO", "DAR", "DER", "EST-L", "EST-S", "FUL", "GRA",
        "HER", "IRO", "JAC", "LAT", "MLT", "MOB", "THM", "URB", "WDC"
    ]

    url = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3/query"

    print(f"\n--- Run with harvest_status: {include_harvest_status} ---\n")

    for loc in locations:
        # Determine fields to query
        if include_harvest_status:
            fields = "report_location,report_tract_no,Tract_Name,tract_status_desc,Forester,tract_type_family,SaleType,latitude_dd,longitude_dd,Wthr_grd,PurchDate,harvest_status"
        else:
            fields = "report_location,report_tract_no,Tract_Name,tract_status_desc,Forester,tract_type_family,SaleType,latitude_dd,longitude_dd,Wthr_grd,PurchDate"

        params = {
            "where": f"report_location='{loc}'",
            "outFields": fields,
            "resultRecordCount": 1000,
            "f": "json",
            "returnGeometry": "false"
        }

        # Count request
        count_params = {
            "where": f"report_location='{loc}'",
            "returnCountOnly": "true",
            "f": "json"
        }

        count_resp = requests.get(url, params=count_params)
        count = count_resp.json().get("count", 0)

        # Timed request
        start = time.time()
        response = requests.get(url, params=params)
        elapsed = round((time.time() - start) * 1000)  # in milliseconds

        # Debug the response
        resp_json = response.json()

        # Check for error in response
        if "error" in resp_json:
            error_info = resp_json["error"]
            print(f'{{ "report_location": "{loc}", "total_count": {count}, "error": {json.dumps(error_info)}, "time": "{elapsed:,} ms" }}')
            continue

        # Get actual features returned
        features = resp_json.get("features", [])
        features_count = len(features)

        # Print first record structure for debugging if available
        sample = ""
        if features_count > 0:
            sample = json.dumps(features[0], indent=None)[:100] + "..."

        print(f'{{ "report_location": "{loc}", "total_count": {count}, "returned_count": {features_count}, "time": "{elapsed:,} ms", "sample": "{sample}" }}')

# Run twice - first with harvest_status, then without
run_query(include_harvest_status=True)
run_query(include_harvest_status=False)


--- Run with harvest_status: True ---

{ "report_location": "AXI", "total_count": 120, "error": {"code": 400, "message": "Failed to execute query.", "details": []}, "time": "964 ms" }
{ "report_location": "CAM", "total_count": 875, "error": {"code": 400, "message": "Failed to execute query.", "details": []}, "time": "949 ms" }
{ "report_location": "CON", "total_count": 883, "error": {"code": 400, "message": "Failed to execute query.", "details": []}, "time": "1,004 ms" }


KeyboardInterrupt: 

In [5]:
import requests
import time
import json

def run_query(include_harvest_status=True):
    # Just test a few locations to troubleshoot
    locations = ["AXI", "CAM", "CON"]

    url = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3/query"

    print(f"\n--- Run with harvest_status: {include_harvest_status} ---\n")

    for loc in locations:
        # Determine fields to query - simplified
        if include_harvest_status:
            fields = "*"  # Request all fields as a test
        else:
            fields = "report_location,report_tract_no,Tract_Name"  # Just a few basic fields

        # Simplify the parameters
        params = {
            "where": f"report_location='{loc}'",
            "outFields": fields,
            "f": "json"
        }

        # First, test if we can get a count
        count_params = {
            "where": f"report_location='{loc}'",
            "returnCountOnly": "true",
            "f": "json"
        }

        try:
            count_resp = requests.get(url, params=count_params)
            count_resp.raise_for_status()  # Check for HTTP errors
            count_data = count_resp.json()
            count = count_data.get("count", 0)

            # Show full response for debugging
            print(f"Count response for {loc}: {json.dumps(count_data)[:200]}...")
        except Exception as e:
            print(f"Error getting count for {loc}: {str(e)}")
            continue

        # Now try a basic query
        try:
            start = time.time()
            response = requests.get(url, params=params)
            response.raise_for_status()  # Check for HTTP errors
            elapsed = round((time.time() - start) * 1000)

            resp_data = response.json()

            # Check for API error
            if "error" in resp_data:
                print(f'{{ "report_location": "{loc}", "error": {json.dumps(resp_data["error"])}, "time": "{elapsed:,} ms" }}')
                continue

            # Get features count
            features = resp_data.get("features", [])
            features_count = len(features)

            # Show sample of response for debugging
            print(f"Query response structure: {json.dumps(resp_data)[:200]}...")

            print(f'{{ "report_location": "{loc}", "total_count": {count}, "returned_count": {features_count}, "time": "{elapsed:,} ms" }}')

            # If we have features, show first one
            if features_count > 0:
                print(f"Sample feature: {json.dumps(features[0])[:200]}...")
        except Exception as e:
            print(f"Error querying {loc}: {str(e)}")

# Try with just the first approach to troubleshoot
run_query(include_harvest_status=True)


--- Run with harvest_status: True ---

Count response for AXI: {"count": 120}...
Query response structure: {"displayFieldName": "Tract_Name", "fieldAliases": {"Location": "Location", "primary_loc": "primary_loc", "state_location": "state_location", "Tract_No": "Tract_No", "report_tract_no": "report_tract_n...
{ "report_location": "AXI", "total_count": 120, "returned_count": 120, "time": "2,001 ms" }
Sample feature: {"attributes": {"Location": "AXI", "primary_loc": "Axis", "state_location": "AL", "Tract_No": 73244, "report_tract_no": 73244, "report_location": "AXI", "tract_relationship": "Parent", "Tract_Name": "...
Count response for CAM: {"count": 875}...
Query response structure: {"displayFieldName": "Tract_Name", "fieldAliases": {"Location": "Location", "primary_loc": "primary_loc", "state_location": "state_location", "Tract_No": "Tract_No", "report_tract_no": "report_tract_n...
{ "report_location": "CAM", "total_count": 875, "returned_count": 875, "time": "77,046 ms" }
Sample feat

KeyboardInterrupt: 

In [7]:
import requests
import time
import json

def run_query(include_harvest_status=True):
    locations = [
        "AXI", "CAM", "CON", "CRO", "DAR", "DER", "EST-L", "EST-S", "FUL", "GRA",
        "HER", "IRO", "JAC", "LAT", "MLT", "MOB", "THM", "URB", "WDC"
    ]

    url = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3/query"

    print(f"\n--- Run with harvest_status: {include_harvest_status} ---\n")

    for loc in locations:
        # Determine fields to query
        if include_harvest_status:
            fields = "report_location,report_tract_no,Tract_Name,tract_status_desc,Forester,tract_type_family,SaleType,latitude_dd,longitude_dd,Wthr_grd,PurchDate,harvest_status"
        else:
            fields = "report_location,report_tract_no,Tract_Name,tract_status_desc,Forester,tract_type_family,SaleType,latitude_dd,longitude_dd,Wthr_grd,PurchDate"

        # Basic parameters - no resultRecordCount
        params = {
            "where": f"report_location='{loc}'",
            "outFields": fields,
            "f": "json"
        }

        # Count request
        count_params = {
            "where": f"report_location='{loc}'",
            "returnCountOnly": "true",
            "f": "json"
        }

        try:
            # Get count
            count_resp = requests.get(url, params=count_params)
            count_resp.raise_for_status()
            count = count_resp.json().get("count", 0)

            # Limit to 300 records if more are available by adding OBJECTID filter
            if count > 300:
                params["where"] = f"report_location='{loc}' AND OBJECTID <= (SELECT MIN(OBJECTID) + 300 FROM {url.split('/')[-2]} WHERE report_location='{loc}')"

            # Actual query
            start = time.time()
            response = requests.get(url, params=params)
            response.raise_for_status()
            elapsed = round((time.time() - start) * 1000)

            resp_data = response.json()

            # Check for API error
            if "error" in resp_data:
                print(f'{{ "report_location": "{loc}", "error": "{resp_data["error"]["message"]}", "time": "{elapsed:,} ms" }}')
                continue

            # Get features count
            features_count = len(resp_data.get("features", []))

            # Cap at 300 for reporting
            effective_count = min(count, 300)

            # Print concise result
            print(f'{{ "report_location": "{loc}", "total_count": {count}, "returned_count": {features_count}, "time": "{elapsed:,} ms" }}')

        except Exception as e:
            print(f'{{ "report_location": "{loc}", "error": "{str(e)}", "time": "N/A" }}')

# Run both scenarios
run_query(include_harvest_status=True)
run_query(include_harvest_status=False)


--- Run with harvest_status: True ---

{ "report_location": "AXI", "total_count": 120, "returned_count": 120, "time": "1,905 ms" }
{ "report_location": "CAM", "error": "Unable to complete operation.", "time": "390 ms" }
{ "report_location": "CON", "error": "Unable to complete operation.", "time": "478 ms" }
{ "report_location": "CRO", "total_count": 29, "returned_count": 29, "time": "1,325 ms" }
{ "report_location": "DAR", "error": "Unable to complete operation.", "time": "408 ms" }
{ "report_location": "DER", "total_count": 160, "returned_count": 160, "time": "2,498 ms" }
{ "report_location": "EST-L", "error": "Unable to complete operation.", "time": "426 ms" }
{ "report_location": "EST-S", "total_count": 194, "returned_count": 194, "time": "4,055 ms" }


KeyboardInterrupt: 

In [9]:
"""
ArcGIS REST API Query Performance Analyzer
=========================================

A utility script for testing and comparing performance of ArcGIS REST API queries
with different field selections.

This script tests response times when querying the Canfor Woodpro database,
comparing performance with and without the harvest_status field included.

Usage:
------
1. Run the script as-is to test all locations with both query scenarios
2. Modify the locations list to focus on specific regions
3. Update field lists if different fields need to be compared
4. Adjust the record limit threshold (currently 500) as needed

Requirements:
------------
- Python 3.6+
- requests library

Example Output:
--------------
--- Run with harvest_status: True ---
{ "report_location": "AXI", "total_count": 120, "returned_count": 120, "time": "1,905 ms" }
{ "report_location": "CRO", "total_count": 29, "returned_count": 29, "time": "1,325 ms" }

Summary for with harvest_status:
Total successful queries: 10
Average query time: 2,345.6 ms
Total time: 23,456.0 ms

Notes:
-----
- Large datasets (>500 records) are skipped to avoid timeouts
- "Unable to complete operation" errors usually indicate server resource limitations
- Consider implementing pagination for comprehensive analysis of large datasets

Author: [Your Name]
Date: [Current Date]
Version: 1.0
"""

import requests
import time
import json

def run_query(include_harvest_status=True):
    # Use the locations that seemed to work
    locations = [
        "AXI", "CRO", "DER", "EST-S", "FUL", "GRA",
        "HER", "IRO", "JAC", "LAT", "MLT", "MOB", "THM", "URB", "WDC"
    ]

    url = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3/query"

    print(f"\n--- Run with harvest_status: {include_harvest_status} ---\n")

    results = []

    for loc in locations:
        # Determine fields to query
        if include_harvest_status:
            fields = "report_location,report_tract_no,Tract_Name,tract_status_desc,Forester,tract_type_family,SaleType,latitude_dd,longitude_dd,Wthr_grd,PurchDate,harvest_status"
        else:
            fields = "report_location,report_tract_no,Tract_Name,tract_status_desc,Forester,tract_type_family,SaleType,latitude_dd,longitude_dd,Wthr_grd,PurchDate"

        # Basic parameters
        params = {
            "where": f"report_location='{loc}'",
            "outFields": fields,
            "returnGeometry": "false",
            "f": "json"
        }

        # Count request
        count_params = {
            "where": f"report_location='{loc}'",
            "returnCountOnly": "true",
            "f": "json"
        }

        try:
            # Get count
            count_resp = requests.get(url, params=count_params)
            count_resp.raise_for_status()
            count = count_resp.json().get("count", 0)

            # Skip if more than 500 records to avoid timeouts
            if count > 500:
                print(f'{{ "report_location": "{loc}", "total_count": {count}, "status": "skipped - too many records", "time": "N/A" }}')
                continue

            # Actual query
            start = time.time()
            response = requests.get(url, params=params)
            response.raise_for_status()
            elapsed = round((time.time() - start) * 1000)

            resp_data = response.json()

            # Check for API error
            if "error" in resp_data:
                print(f'{{ "report_location": "{loc}", "error": "{resp_data["error"]["message"]}", "time": "{elapsed:,} ms" }}')
                continue

            # Get features count
            features_count = len(resp_data.get("features", []))

            # Print concise result
            result = {
                "report_location": loc,
                "total_count": count,
                "returned_count": features_count,
                "time": elapsed
            }

            results.append(result)
            print(f'{{ "report_location": "{loc}", "total_count": {count}, "returned_count": {features_count}, "time": "{elapsed:,} ms" }}')

        except Exception as e:
            print(f'{{ "report_location": "{loc}", "error": "{str(e)}", "time": "N/A" }}')

    # Print summary if we got results
    if results:
        total_time = sum(r["time"] for r in results)
        avg_time = total_time / len(results)
        print(f"\nSummary for {'with' if include_harvest_status else 'without'} harvest_status:")
        print(f"Total successful queries: {len(results)}")
        print(f"Average query time: {avg_time:,.1f} ms")
        print(f"Total time: {total_time:,.1f} ms")

# Run both scenarios
run_query(include_harvest_status=True)
run_query(include_harvest_status=False)


--- Run with harvest_status: True ---

{ "report_location": "AXI", "total_count": 120, "returned_count": 120, "time": "1,803 ms" }
{ "report_location": "CRO", "total_count": 29, "returned_count": 29, "time": "1,273 ms" }
{ "report_location": "DER", "total_count": 160, "returned_count": 160, "time": "2,302 ms" }


KeyboardInterrupt: 

In [ ]:
import requests

URL = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3/query"
METADATA_URL = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3?f=pjson"

TEST_LOC = "CAM"
BASE_FIELD = "report_location"
MAX_RECORDS = 1

def get_all_fields():
    res = requests.get(METADATA_URL)
    res.raise_for_status()
    return [field["name"] for field in res.json().get("fields", [])]

def test_field_combo(fields):
    params = {
        "where": f"UPPER({BASE_FIELD})='{TEST_LOC}'",
        "outFields": ",".join(fields),
        "returnGeometry": "false",
        "resultRecordCount": str(MAX_RECORDS),
        "f": "json"
    }
    try:
        response = requests.get(URL, params=params)
        data = response.json()
        return len(data.get("features", [])) > 0
    except Exception as e:
        print(f"⚠️ Error for fields {fields}: {e}")
        return False

def main():
    print("🔍 Fetching all fields...")
    all_fields = get_all_fields()
    print("\n🧾 Field names from schema:\n")
    for f in all_fields:
        print(f)

    print(f"\n🧪 Testing {len(all_fields)-1} fields with base field '{BASE_FIELD}' (using location='{TEST_LOC}'):\n")

    working_fields = []
    for field in all_fields:
        if field == BASE_FIELD:
            continue
        combo = [BASE_FIELD, field]
        if test_field_combo(combo):
            print(f"✅ Works with: {field}")
            working_fields.append(field)
        else:
            print(f"❌ Fails with: {field}")

    print("\n✅ Summary:")
    print(f"Total working fields with {BASE_FIELD}: {len(working_fields)}")
    print("Working fields:")
    for field in working_fields:
        print(f"  - {field}")

    # Optional: save to file
    with open("working_fields.txt", "w") as f:
        f.write(",".join([BASE_FIELD] + working_fields))
        f.write("\n")
    print("\n📝 Saved to 'working_fields.txt' as a comma-delimited list.")

if __name__ == "__main__":
    main()


In [ ]:
import requests

URL = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3/query"
METADATA_URL = "https://maps.canfor.com/arcgis/rest/services/CSPWoodpro/WoodPro_NSApps_DB_Views/MapServer/3?f=pjson"

TEST_LOC = "CAM"
BASE_FIELD = "report_location"
MAX_RECORDS = 1

def get_all_fields():
    response = requests.get(METADATA_URL)
    response.raise_for_status()
    return [field["name"] for field in response.json().get("fields", [])]

def test_field_combo(fields):
    out_fields = ",".join(fields)
    params = {
        "where": f"UPPER({BASE_FIELD})='{TEST_LOC}'",
        "outFields": out_fields,
        "returnGeometry": "false",
        "resultRecordCount": str(MAX_RECORDS),
        "f": "json"
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    print(f"\n🔍 Testing combo: {out_fields}")
    response = requests.post(URL, data=params, headers=headers)
    print(f"🌐 URL: {response.url}")
    print(f"📦 Response:\n{response.text[:500]}...\n")

    try:
        data = response.json()
        features = data.get("features", [])
        return len(features) > 0
    except Exception as e:
        print(f"⚠️ Error testing fields {fields}: {e}")
        return False

def main():
    print("🔍 Fetching field list...")
    all_fields = get_all_fields()
    print("\n🧾 Field names from schema:\n")
    for f in all_fields:
        print(f)

    print(f"\n🧪 Testing field combinations with '{BASE_FIELD}' using report_location='{TEST_LOC}':\n")

    working_fields = []
    for field in all_fields:
        if field == BASE_FIELD:
            continue
        combo = [BASE_FIELD, field]
        if test_field_combo(combo):
            print(f"✅ Works with: {field}")
            working_fields.append(field)
        else:
            print(f"❌ Fails with: {field}")

    print("\n✅ Working fields:")
    for f in working_fields:
        print(f"  - {f}")

    print("\n💾 Saving to working_fields.txt...")
    with open("working_fields.txt", "w") as f:
        f.write(",".join([BASE_FIELD] + working_fields) + "\n")
    print("Done.")

if __name__ == "__main__":
    main()
